In [ ]:
!pip install pytorch-lightning #==1.2.8 --quiet
!pip install transformers #==4.5.1 --quiet
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 24.1 MB/s 
     |████████████████████████████████| 419 kB 74.4 MB/s 
     |████████████████████████████████| 140 kB 78.2 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.6 MB/s 
     |████████████████████████████████| 144 kB 63.9 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 271 kB 61.9 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.9 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 34.7 MB/s 
Looking in indexes: 

In [ ]:
#pytorch libraries 
import pytorch_lightning as pl
try:
    from torchmetrics.functional import accuracy, f1_score as f1, auroc
except ImportError:
    from pytorch_lightning.metrics.functional import accuracy, f1, auroc
#from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaTokenizerFast , BertConfig, BertModel,RobertaModel, AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, plot_precision_recall_curve, precision_recall_curve, confusion_matrix
from sklearn.metrics import roc_curve, auc

#Tensorflow libraries
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, concatenate, GlobalMaxPool1D, Conv1D, LSTM, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from itertools import cycle

import numpy as np
import pandas as pd
import re
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import subprocess
import time
import datetime
import timeit

#import custom modules

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/')
import tf_models, data_utils, utils
from init import pt_initialize, tf_initialize
from dga_transformer import DGADataset, DGADataModule, DGATransformer

#jupyter setup
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

#initialization
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)
root = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/'
related_models = ['highnam.h5', 'lstm1_mi.h5', 'n-CDBC.h5']


Global seed set to 42
Global seed set to 42


In [6]:
LABELS_NO = 21
amrita = data_utils.read_amrita(f'{root}data/multi', 'tf', labels = LABELS_NO)
train_df, val_df, test_df = data_utils.read_amrita(f'{root}data/multi', labels = LABELS_NO)
umudga_df = data_utils.read_data(f'{root}data/multi/biumudga_test.csv', labels = LABELS_NO)
unknown_df = data_utils.read_data(f'{root}data/multi/unknown_dga.csv', labels =  LABELS_NO)
charbot_df = data_utils.read_data(f'{root}data/multi/charbotsamples.csv', labels =  LABELS_NO)

#read the alexa top 1 million non dga websites
alexa = data_utils.read_data(f'{root}data/multi/alexa.csv', labels = LABELS_NO)
#split alexa for training and testing
alexa_test = alexa[0:15000]
alexa_train = alexa[15000:]

In [ ]:
test_df.head()

In [7]:
LABELS_NO = 21
amrita = data_utils.read_amrita(f'{root}data/multi', 'tf', labels = LABELS_NO)
train_df, val_df, test_df = data_utils.read_amrita(f'{root}data/multi', labels = LABELS_NO)
umudga_df = data_utils.read_data(f'{root}data/multi/biumudga_test.csv', labels = LABELS_NO)
unknown_df = data_utils.read_data(f'{root}data/multi/unknown_dga.csv', labels =  LABELS_NO)
charbot_df = data_utils.read_data(f'{root}data/multi/charbotsamples.csv', labels =  LABELS_NO)

#read the alexa top 1 million non dga websites
alexa = data_utils.read_data(f'{root}data/multi/alexa.csv', labels = LABELS_NO)
#split alexa for training and testing
alexa_test = alexa[0:15000]
alexa_train = alexa[15000:]

In [8]:
train_df['benign'].value_counts(), val_df['benign'].value_counts(), test_df['benign'].value_counts()

(0    241265
 1     80934
 Name: benign, dtype: int64, 0    26884
 1     8916
 Name: benign, dtype: int64, 0    29628
 1    10150
 Name: benign, dtype: int64)

In [9]:
umudga_df['benign'].value_counts(), unknown_df['benign'].value_counts(), alexa_test['benign'].value_counts()

(0    15000
 1    10000
 Name: benign, dtype: int64, 0    15214
 Name: benign, dtype: int64, 1    15000
 Name: benign, dtype: int64)

In [ ]:
.

# **Amrita Dataset**

In [ ]:
amrita = umudga_df

In [ ]:
LABEL_COLUMNS = amrita.columns.tolist() # 0 for DGA and 1 for benign
LABEL_COLUMNS

In [ ]:
LABELS_NO = 21
#store testing data
training_stats = []
testing_record = []
THRESHOLD, upper, lower = 0.5, 1 , 0 
device_name = utils.get_tf_device()
test_time = time.time() 

#Compute ROC curve and ROC area
def get_tf_fpr_fnr(y_pred, y_test, n_classes=1):
  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_test, y_pred)
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_test, y_pred)
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  #print('Micro = ', roc_auc["micro"])

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
  lw = 2
  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(n_classes):
      mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= n_classes

  fpr["macro"] = all_fpr
  tpr["macro"] = mean_tpr
  roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])


  return fpr, tpr, roc_auc

#compute dga-transformer tpr and fpr
def get_pt_fpr_tpr(y_pred, y_test, n_classes = 1):
  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_pred[:, i])
      roc_auc[i] = auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
  lw = 2
  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(n_classes):
      mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= n_classes

  fpr["macro"] = all_fpr
  tpr["macro"] = mean_tpr
  roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

  return fpr, tpr, roc_auc

  

def add_stats(model_name, dataset, acc, prec, fscore, recall, test_time, DGAs, benign, aucurve):
  # Record all statistics from this epoch.
  training_stats.append(
      {
          'Model_Name':model_name,
          'Dataset': dataset,
          'Test Time': test_time,
          'Accuracy': acc,
          'Precision': prec,
          'Recall': recall,
          'F Score': fscore,
          'AUROC': aucurve,
          'DGA Samples':DGAs,
          'Benign Samples': benign
      }
  )


def test_model(model, X_test):
  t0 = time.time() 
  predictions = model.predict(X_test, verbose=True)
  test_time = utils.format_time(time.time() - t0)
  #yhat = model.predict_classes(X_test)
  #yhat = model.predict_proba(X_test)  
  return predictions.flatten(), test_time

#load and test the

#prepare the character based tokenizer
tk, MAX_INDEX = tf_models.prepare_tokenizer(maxlen = tf_initialize.MAX_STRING_LENGTH)
X, y, class_list, classes = tf_models.text_2_sequence(amrita, tk, tf_initialize.MAX_STRING_LENGTH, LABELS_NO = LABELS_NO)

#split data into train, test, val
X_train, X_test_, y_train, y_test_ = train_test_split(X, y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_, y_test_, test_size=0.50, random_state=42)

  


In [ ]:
def format_prediction(prediction, y_test_len, LABELS_NO):
  i, j = 0,0
  y_pred = [] 
  while (j < y_test_len):
    j = j + LABELS_NO
    y_pred.append(prediction[i:j])
    i = i+ LABELS_NO
  return np.array(y_pred)

In [ ]:

#start testing
total_stats = {}
#with tf.device(device_name):
model_name = 'highnam.h5'
high_model = tf_models.build_highnam(n_classes = LABELS_NO)

#load the pretrained model    
high_trained_model =  tf.keras.models.load_model(f'{root}models/multi/{model_name}')
predictions, high_test_time = test_model(high_trained_model, X_test)
predictions = np.where(predictions > THRESHOLD, upper, lower)
high_y_pred = format_prediction(predictions, predictions.shape[0], LABELS_NO)

#Confusion Matrix
#high_conf_matrix = confusion_matrix(y_test, high_y_pred)

#Accuracy
high_acc = accuracy_score(y_test, high_y_pred)
#Compute AUROC
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS[1:]):
  high_tag_auroc = roc_auc_score(y_test[:, i], high_y_pred[:, i])
  print(f"{name}: {high_tag_auroc}")


#Compute  classification report 
high_class_rpt = classification_report(y_test, high_y_pred, digits=4, output_dict=False)
#print(high_conf_matrix)
print(high_acc)
print(high_class_rpt)

In [ ]:
high_final_auroc = auroc(torch.tensor(high_y_pred), torch.tensor(y_test), num_classes= LABELS_NO, average='macro')
high_final_auroc

In [ ]:
tf_initialize.MAX_INDEX

In [ ]:
tf_initialize.MAX_STRING_LENGTH

In [ ]:
.

In [ ]:
model_name = 'lstm1_mi.h5'
m1_model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH, n_classes = 21)

#load the pretrained model    
m1_trained_model =  tf.keras.models.load_model(f'{root}models/multi/{model_name}')
predictions, m1_test_time = test_model(m1_trained_model, X_test)
predictions = np.where(predictions > THRESHOLD, upper, lower)
m1_y_pred = format_prediction(predictions, predictions.shape[0], LABELS_NO)

#Confusion Matrix
#high_conf_matrix = confusion_matrix(y_test, high_y_pred)

#Accuracy
m1_acc = accuracy_score(y_test, m1_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS[1:]):
  m1_tag_auroc = roc_auc_score(y_test[:, i], m1_y_pred[:, i])
  print(f"{name}: {m1_tag_auroc}")


#Compute  classification report 
m1_class_rpt = classification_report(y_test, m1_y_pred, digits=4, output_dict=False, zero_division=0)
#print(high_conf_matrix)
print(m1_acc)
print(m1_class_rpt)

In [ ]:
m1_final_auroc = auroc(torch.tensor(m1_y_pred), torch.tensor(y_test), num_classes= LABELS_NO, average='macro')
m1_final_auroc

In [ ]:
model_name = 'n-CDBC.h5'
CDBC_model = tf_models.build_highnam(n_classes = LABELS_NO)

#load the pretrained model    
CDBC_trained_model =  tf.keras.models.load_model(f'{root}models/multi/{model_name}')
predictions, CDBC_test_time = test_model(CDBC_trained_model, X_test)
predictions = np.where(predictions > THRESHOLD, upper, lower)
CDBC_y_pred = format_prediction(predictions, predictions.shape[0], LABELS_NO)

#Confusion Matrix
#high_conf_matrix = confusion_matrix(y_test, high_y_pred)

#Accuracy
CDBC_acc = accuracy_score(y_test, CDBC_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS[1:]):
  CDBC_tag_auroc = roc_auc_score(y_test[:, i], CDBC_y_pred[:, i])
  print(f"{name}: {CDBC_tag_auroc}")


#Compute  classification report 
CDBC_class_rpt = classification_report(y_test, CDBC_y_pred, digits=4, output_dict=False)
#print(high_conf_matrix)
print(CDBC_acc)
print(CDBC_class_rpt)

In [ ]:
CBDC_final_auroc = auroc(torch.tensor(CDBC_y_pred), torch.tensor(y_test), num_classes= LABELS_NO, average='macro')
CBDC_final_auroc

**DGA Transformer**

In [ ]:
checkpoint = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/logs/lightning_logs/multi/domain_name/version_0/checkpoints/epoch=4-step=7815.ckpt'
THRESHOLD = 0.5
dga_trained_model = DGATransformer.load_from_checkpoint(
    checkpoint, 
    LABEL_COLUMNS = test_df.columns.tolist()[1:]
)
dga_trained_model.eval()
dga_trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#deine the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(pt_initialize.MODEL_NAME, max_len=pt_initialize.MAX_LENGTH)

In [ ]:

dga_trained_model = dga_trained_model.to(device)
LABELS = test_df.columns.tolist()[1:]
test_dataset = DGADataset(
  test_df,
  tokenizer,
  max_token_len=pt_initialize.MAX_LENGTH,
  categories = LABELS
)

dga_predictions = []
dga_labels = []

t0 = time.time()
for item in tqdm(test_dataset):
  _, prediction = dga_trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  dga_predictions.append(prediction.flatten())
  dga_labels.append(item["labels"].int())

dga_test_time = utils.format_time(time.time() - t0)


dga_predictions = torch.stack(dga_predictions).detach().cpu()
dga_labels = torch.stack(dga_labels).detach().cpu()


In [ ]:
dga_acc = accuracy(dga_predictions, dga_labels, threshold=THRESHOLD)
dga_y_pred = dga_predictions.numpy()
y_true = dga_labels.numpy()

upper, lower = 1, 0
dga_y_pred = np.where(dga_y_pred > THRESHOLD, upper, lower)
dga_class_rpt = classification_report(y_true, dga_y_pred, digits=4)

#dga_conf_matrix = confusion_matrix(y_true, dga_y_pred)

for i, name in enumerate(test_df.columns.tolist()[1:]):
  dga_tag_auroc = auroc(dga_predictions[:, i], dga_labels[:, i], pos_label=1)
  print(f"{name}: {dga_tag_auroc}")
#print(dga_conf_matrix)
print(dga_acc)
print(dga_class_rpt)
#print(dga_tag_auroc)

In [ ]:
dga_final_auroc = auroc(dga_predictions, dga_labels, num_classes= LABELS_NO, average='macro')
dga_final_auroc

In [ ]:
jhgjhf

In [ ]:
scores = []
for i in high_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('Highnam', 'Amrita DGA', round(high_acc.item(),3), prec, fscore, recall, high_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(high_final_auroc.item(),3))

In [ ]:

#with tf.device(device_name):
model_name = 'lstm1_mi.h5'
m1_model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH, n_classes = LABELS_NO)

#load the pretrained model    
m1_trained_model =  tf.keras.models.load_model(f'{root}models/multi/{model_name}')
m1_predictions, m1_test_time = test_model(m1_trained_model, X_test)
m1_y_pred = np.where(m1_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
m1_conf_matrix = confusion_matrix(y_test, m1_y_pred)

#Accuracy
m1_acc = accuracy_score(y_test, m1_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  m1_tag_auroc = roc_auc_score(y_test, m1_y_pred)
  print(f"{name}: {m1_tag_auroc}")

#Compute  classification report 
m1_class_rpt = classification_report(y_test, m1_y_pred, digits=4)
print(m1_conf_matrix)
print(m1_acc)
print(m1_class_rpt)

In [ ]:
m1_final_auroc = auroc(m1_y_pred, y_test, num_classes= LABELS_NO, average='macro')
m1_final_auroc


In [ ]:
scores = []
for i in m1_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('LSTM1.MI', 'Amrita DGA', round(m1_acc.item(),3), prec, fscore, recall, m1_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(m1_tag_auroc.item(),3))

In [ ]:

#with tf.device(device_name):
model_name = 'n-CDBC.h5'
CBDC_model = tf_models.build_CBDC(n_classes = LABELS_NO)

#load the pretrained model    
CBDC_trained_model =  tf.keras.models.load_model(f'{root}models/multi/{model_name}')
CBDC_predictions, CBDC_test_time = test_model(CBDC_trained_model, X_test)
CBDC_y_pred = np.where(CBDC_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
CBDC_conf_matrix = confusion_matrix(y_test, CBDC_y_pred)

#Accuracy
CBDC_acc = accuracy_score(y_test, CBDC_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  CBDC_tag_auroc = roc_auc_score(y_test, CBDC_y_pred)
  print(f"{name}: {CBDC_tag_auroc}")

#Compute  classification report 
CBDC_class_rpt = classification_report(y_test, CBDC_y_pred, digits=4)

print(CBDC_conf_matrix)
print(CBDC_acc)
print(CBDC_class_rpt)

In [ ]:
CBDC_final_auroc = auroc(high_y_pred, y_test, num_classes= LABELS_NO, average='macro')
CBDC_final_auroc


In [ ]:
scores = []
for i in CBDC_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('n-CBDC', 'Amrita DGA', round(CBDC_acc.item(),3), prec, fscore, recall, CBDC_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(CBDC_tag_auroc.item(),3))

**Test DGATransformers**

In [ ]:
#deine the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained(pt_initialize.MODEL_NAME, max_len=pt_initialize.MAX_LENGTH)

#define data
data_module = DGADataModule(
  train_df,
  val_df,
  test_df,
  tokenizer,
  train_batch_size=pt_initialize.TRAIN_BATCH_SIZE,
  val_batch_size= pt_initialize.EVAL_BATCH_SIZE,
  test_batch_size= pt_initialize.EVAL_BATCH_SIZE,
  max_token_len= pt_initialize.MAX_LENGTH, 
  LABELS = train_df.columns.tolist()[1:]
)

#Optimizer scheduler
steps_per_epoch=len(test_df) // pt_initialize.EVAL_BATCH_SIZE
total_testing_steps = steps_per_epoch * pt_initialize.MAX_EPOCH

#We'll use a fifth of the training steps for a warm-up:
warmup_steps = total_testing_steps // 5

#define the model
#model = DGATransformer(test_df.columns.tolist()[1:],n_warmup_steps=warmup_steps, n_training_steps=total_testing_steps )
#Initialize Trainer
#trainer = pl.Trainer(gpus= pt_initialize.AVAIL_GPUS)

In [ ]:
checkpoint = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/logs/lightning_logs/multi/domain_name/version_0/checkpoints/epoch=4-step=7815.ckpt'
THRESHOLD = 0.5
dga_trained_model = DGATransformer.load_from_checkpoint(
    checkpoint, 
    LABEL_COLUMNS = test_df.columns.tolist()[1:]
)
dga_trained_model.eval()
dga_trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dga_trained_model = dga_trained_model.to(device)
LABELS = test_df.columns.tolist()[1:]
test_dataset = DGADataset(
  test_df,
  tokenizer,
  max_token_len=pt_initialize.MAX_LENGTH,
  categories = LABELS
)

dga_predictions = []
dga_labels = []

t0 = time.time()
for item in tqdm(test_dataset):
  _, prediction = dga_trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  dga_predictions.append(prediction.flatten())
  dga_labels.append(item["labels"].int())

dga_test_time = utils.format_time(time.time() - t0)


dga_predictions = torch.stack(dga_predictions).detach().cpu()
dga_labels = torch.stack(dga_labels).detach().cpu()


In [ ]:


dga_acc = accuracy(dga_predictions, dga_labels, threshold=THRESHOLD)
dga_y_pred = dga_predictions.numpy()
y_true = dga_labels.numpy()

upper, lower = 1, 0
dga_y_pred = np.where(dga_y_pred > THRESHOLD, upper, lower)
dga_class_rpt = classification_report(y_true, dga_y_pred, digits=4)

dga_conf_matrix = confusion_matrix(y_true, dga_y_pred)

for i, name in enumerate(test_df.columns.tolist()[1:]):
  dga_tag_auroc = auroc(dga_predictions[:, i], dga_labels[:, i], pos_label=1)

print(dga_conf_matrix)
print(dga_acc)
print(dga_class_rpt)
print(dga_tag_auroc)

In [ ]:
dga_final_auroc = auroc(predictions, labels, num_classes= n_classes, average='macro')
dga_final_auroc


In [ ]:
scores = []
for i in dga_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('DGA-Transformer','Amrita DGA', round(dga_acc.item(),3), prec, fscore, recall, dga_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(dga_tag_auroc.item(),3))

**Final Results**

In [ ]:
stats_df = utils.result_df(training_stats)
stats_df

In [ ]:
stats_df.to_csv('/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/amrita-6-6-2022-3layers.csv')

**Draw Figures**

In [ ]:
image_path = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/images'

#get base models values
high_fpr, high_tpr, high_roc_auc = get_tf_fpr_fnr(high_predictions, y_test)
m1_fpr, m1_tpr, m1_roc_auc = get_tf_fpr_fnr(m1_predictions, y_test)
CBDC_fpr, CBDC_tpr, CBDC_roc_auc = get_tf_fpr_fnr(CBDC_predictions, y_test)

#get DGA_transformer values
dga_fpr, dga_tpr, dga_roc_auc = get_pt_fpr_tpr(dga_predictions.numpy(), dga_labels.numpy())


In [ ]:
dga_roc_auc['macro'], high_roc_auc['micro']

In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["micro"],
    high_tpr["micro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["micro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["micro"],
    m1_tpr["micro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["micro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["micro"],
    CBDC_tpr["micro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["micro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["micro"],
    dga_tpr["micro"],
    label="DGA-Trans Micro area = {0:0.3f}".format(dga_roc_auc["micro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Micro) AmritaDGA Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/amrita_micro.png",dpi=300)

In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["macro"],
    high_tpr["macro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["macro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["macro"],
    m1_tpr["macro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["macro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["macro"],
    CBDC_tpr["macro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["macro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["macro"],
    dga_tpr["macro"],
    label="DGA-Trans Macro area = {0:0.3f}".format(dga_roc_auc["macro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Macro) AmritaDGA Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/amrita_macro.png",dpi=300)

In [ ]:
n_classes = len(test_df.columns.tolist()[1:])
n_classes

In [ ]:
n_classes = len(test_df.columns.tolist()[1:])
# Plot all ROC curves
plt.figure()
lw = 2
for i in range(n_classes):
  plt.plot(
      high_fpr[0],
      high_tpr[0],
      label="Highnam AUROC = {0:0.3f}".format(high_tag_auroc),
      color="aqua",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      m1_fpr[0],
      m1_tpr[0],
      label="LSTM1.MI AUROC = {0:0.3f}".format(m1_tag_auroc),
      color="darkorange",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      CBDC_fpr[0],
      CBDC_tpr[0],
      label="CBDC AUROC = {0:0.3f}".format(CBDC_tag_auroc),
      color="cornflowerblue",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      dga_fpr[0],
      dga_tpr[0],
      label="DGA-Trans AUROC = {0:0.3f}".format(dga_tag_auroc),
      color="purple",
      linestyle=":",
      linewidth=4
  )

  plt.plot([0, 1], [0, 1], "k--", lw=lw)
  plt.xlim([0.0, 1.05])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("ROC (AUROC) AmritaDGA Dataset")
  plt.legend(loc="lower right")
  plt.show()
plt.savefig(f"{image_path}/amrita_acc.png",dpi=300)

In [ ]:
# Plot all Precision Recall curves
high_precision, high_recall, _ = precision_recall_curve(y_test, high_y_pred)
m1_precision, m1_recall, _ = precision_recall_curve(y_test, m1_y_pred)
CBDC_precision, CBDC_recall, _ = precision_recall_curve(y_test, CBDC_y_pred)
dga_precision, dga_recall, _ = precision_recall_curve(dga_labels.numpy(), dga_predictions.numpy())
len_benign = len(y_test[y_test==1])
len_dga = len(y_test[y_test==0])



plt.figure()
lw = 2

plt.plot(
    high_precision,
    high_recall,
    label="Highnam",
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_precision,
    m1_recall,
    label="LSTM1.MI",
    color="darkorange",
    linestyle=":",
    linewidth=4
)
plt.plot(
    CBDC_precision,
    CBDC_recall,
    label="n-CBDC",
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)
plt.plot(
    dga_precision,
    dga_recall,
    label="DGA-Transformer",
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [len_benign, len_benign], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve (AmritaDGA)')
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/amrita_prec_recall.png",dpi=300)

In [ ]:
precision = np.nan_to_num(dga_tpr[0]/(dga_tpr[0]+dga_fpr[0]))


# ***umudga Datasets***

In [ ]:
umudga_df = umudga_df.sample(frac = 1).reset_index(drop=True) #shuffle data
umudga_df.head()

In [ ]:
#store testing data
training_stats = []
testing_record = []
THRESHOLD, upper, lower = 0.5, 1 , 0 
device_name = utils.get_tf_device()
test_time = time.time() 
#prepare the character based tokenizer
tk, MAX_INDEX = tf_models.prepare_tokenizer(maxlen = tf_initialize.MAX_STRING_LENGTH)
X, y, class_list, classes = tf_models.text_2_sequence(umudga_df, tk, tf_initialize.MAX_STRING_LENGTH)

X_test = X
y_test = y

Hignnam

In [ ]:

#start testing
total_stats = {}
#with tf.device(device_name):
model_name = 'highnam.h5'
high_model = tf_models.build_highnam()

#load the pretrained model    
high_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
high_predictions, high_test_time = test_model(high_trained_model, X_test)
high_y_pred = np.where(high_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
high_conf_matrix = confusion_matrix(y_test, high_y_pred)

#Accuracy
high_acc = accuracy_score(y_test, high_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  high_tag_auroc = roc_auc_score(y_test, high_y_pred)
  print(f"{name}: {high_tag_auroc}")

#Compute  classification report 
high_class_rpt = classification_report(y_test, high_y_pred, digits=4)
print(high_conf_matrix)
print(high_acc)
print(high_class_rpt)

In [ ]:
scores = []
for i in high_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('Highnam', 'UMU DGA', round(high_acc.item(),3), prec, fscore, recall, high_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(high_tag_auroc.item(),3))

MI

In [ ]:

#with tf.device(device_name):
model_name = 'lstm1_mi.h5'
m1_model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH)

#load the pretrained model    
m1_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
m1_predictions, m1_test_time = test_model(m1_trained_model, X_test)
m1_y_pred = np.where(m1_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
m1_conf_matrix = confusion_matrix(y_test, m1_y_pred)

#Accuracy
m1_acc = accuracy_score(y_test, m1_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  m1_tag_auroc = roc_auc_score(y_test, m1_y_pred)
  print(f"{name}: {m1_tag_auroc}")

#Compute  classification report 
m1_class_rpt = classification_report(y_test, m1_y_pred, digits=4)
print(m1_conf_matrix)
print(m1_acc)
print(m1_class_rpt)

In [ ]:
scores = []
for i in m1_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('LSTM1.MI', 'UMU DGA', round(m1_acc.item(),3), prec, fscore, recall, m1_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(m1_tag_auroc.item(),3))

CDBC

In [ ]:

#with tf.device(device_name):
model_name = 'n-CDBC.h5'
CBDC_model = tf_models.build_CBDC()

#load the pretrained model    
CBDC_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
CBDC_predictions, CBDC_test_time = test_model(CBDC_trained_model, X_test)
CBDC_y_pred = np.where(CBDC_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
CBDC_conf_matrix = confusion_matrix(y_test, CBDC_y_pred)

#Accuracy
CBDC_acc = accuracy_score(y_test, CBDC_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  CBDC_tag_auroc = roc_auc_score(y_test, CBDC_y_pred)
  print(f"{name}: {CBDC_tag_auroc}")

#Compute  classification report 
CBDC_class_rpt = classification_report(y_test, CBDC_y_pred, digits=4)

print(CBDC_conf_matrix)
print(CBDC_acc)
print(CBDC_class_rpt)

In [ ]:
scores = []
for i in CBDC_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('n-CBDC', 'UMU DGA', round(CBDC_acc.item(),3), prec, fscore, recall, CBDC_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(CBDC_tag_auroc.item(),3))

DGA Transformers

In [ ]:
checkpoint = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/logs/lightning_logs/domain_name/version_2/checkpoints/epoch=4-step=7815.ckpt'
THRESHOLD = 0.5
dga_trained_model = DGATransformer.load_from_checkpoint(
    checkpoint, 
    LABEL_COLUMNS = test_df.columns.tolist()[1:]
)
dga_trained_model.eval()
dga_trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dga_trained_model = dga_trained_model.to(device)
LABELS = umudga_df.columns.tolist()[1:]
test_dataset = DGADataset(
  umudga_df,
  tokenizer,
  max_token_len=pt_initialize.MAX_LENGTH,
  categories = LABELS
)

dga_predictions = []
dga_labels = []

t0 = time.time()
for item in tqdm(test_dataset):
  _, prediction = dga_trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  dga_predictions.append(prediction.flatten())
  dga_labels.append(item["labels"].int())

dga_test_time = utils.format_time(time.time() - t0)


dga_predictions = torch.stack(dga_predictions).detach().cpu()
dga_labels = torch.stack(dga_labels).detach().cpu()


In [ ]:


dga_acc = accuracy(dga_predictions, dga_labels, threshold=THRESHOLD)
dga_y_pred = dga_predictions.numpy()
y_true = dga_labels.numpy()

upper, lower = 1, 0
dga_y_pred = np.where(dga_y_pred > THRESHOLD, upper, lower)
dga_class_rpt = classification_report(y_true, dga_y_pred, digits=4)

dga_conf_matrix = confusion_matrix(y_true, dga_y_pred)

for i, name in enumerate(test_df.columns.tolist()[1:]):
  dga_tag_auroc = auroc(dga_predictions[:, i], dga_labels[:, i], pos_label=1)

print(dga_conf_matrix)
print(dga_acc)
print(dga_class_rpt)
print(dga_tag_auroc)

In [ ]:
scores = []
for i in dga_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('DGA-Transformer','UMU DGA', round(dga_acc.item(),3), prec, fscore, recall, dga_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(dga_tag_auroc.item(),3))

In [ ]:
stats_df = utils.result_df(training_stats)
stats_df

In [ ]:
stats_df.to_csv('/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/umu-6-6-2022-3layers.csv')

# **Draw Figures**

In [ ]:
image_path = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/images'

#get base models values
high_fpr, high_tpr, high_roc_auc = get_tf_fpr_fnr(high_predictions, y_test)
m1_fpr, m1_tpr, m1_roc_auc = get_tf_fpr_fnr(m1_predictions, y_test)
CBDC_fpr, CBDC_tpr, CBDC_roc_auc = get_tf_fpr_fnr(CBDC_predictions, y_test)

#get DGA_transformer values
dga_fpr, dga_tpr, dga_roc_auc = get_pt_fpr_tpr(dga_predictions.numpy(), dga_labels.numpy())


In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["micro"],
    high_tpr["micro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["micro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["micro"],
    m1_tpr["micro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["micro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["micro"],
    CBDC_tpr["micro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["micro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["micro"],
    dga_tpr["micro"],
    label="DGA-Trans Micro area = {0:0.3f}".format(dga_roc_auc["micro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Micro) UMUDGA Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/umudga_micro.png",dpi=300)

In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["macro"],
    high_tpr["macro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["macro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["macro"],
    m1_tpr["macro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["macro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["macro"],
    CBDC_tpr["macro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["macro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["macro"],
    dga_tpr["macro"],
    label="DGA-Trans Macro area = {0:0.3f}".format(dga_roc_auc["macro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Macro) UMUDGA Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/umudga_macro.png",dpi=300)

In [ ]:
n_classes = len(test_df.columns.tolist()[1:])
# Plot all ROC curves
plt.figure()
lw = 2
for i in range(n_classes):
  plt.plot(
      high_fpr[0],
      high_tpr[0],
      label="Highnam AUROC = {0:0.3f}".format(high_tag_auroc),
      color="aqua",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      m1_fpr[0],
      m1_tpr[0],
      label="LSTM1.MI AUROC = {0:0.3f}".format(m1_tag_auroc),
      color="darkorange",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      CBDC_fpr[0],
      CBDC_tpr[0],
      label="CBDC AUROC = {0:0.3f}".format(CBDC_tag_auroc),
      color="cornflowerblue",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      dga_fpr[0],
      dga_tpr[0],
      label="DGA-Trans AUROC = {0:0.3f}".format(dga_tag_auroc),
      color="purple",
      linestyle=":",
      linewidth=4
  )

  plt.plot([0, 1], [0, 1], "k--", lw=lw)
  plt.xlim([0.0, 1.05])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("ROC (AUROC) UMUDGA Dataset")
  plt.legend(loc="lower right")
  plt.show()
plt.savefig(f"{image_path}/umudga_auroc.png",dpi=300)

In [ ]:
# Plot all Precision Recall curves
high_precision, high_recall, _ = precision_recall_curve(y_test, high_y_pred)
m1_precision, m1_recall, _ = precision_recall_curve(y_test, m1_y_pred)
CBDC_precision, CBDC_recall, _ = precision_recall_curve(y_test, CBDC_y_pred)
dga_precision, dga_recall, _ = precision_recall_curve(dga_labels.numpy(), dga_predictions.numpy())
len_benign = len(y_test[y_test==1])
len_dga = len(y_test[y_test==0])



plt.figure()
lw = 2

plt.plot(
    high_precision,
    high_recall,
    label="Highnam",
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_precision,
    m1_recall,
    label="LSTM1.MI",
    color="darkorange",
    linestyle=":",
    linewidth=4
)
plt.plot(
    CBDC_precision,
    CBDC_recall,
    label="n-CBDC",
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)
plt.plot(
    dga_precision,
    dga_recall,
    label="DGA-Transformer",
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [len_benign, len_benign], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve (UMUDGA)')
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/umudga_prec_recall.png",dpi=300)

# **Unknown Dataset (DGArchive)**

In [ ]:
unknown_df = pd.concat([
  unknown_df,
  alexa_test.sample(n=14786, random_state = 42)
]).reset_index(drop = True)
unknown_df = unknown_df.sample(frac = 1).reset_index(drop=True) #shuffle data
unknown_df.head()

In [ ]:
#store testing data
training_stats = []
testing_record = []
THRESHOLD, upper, lower = 0.5, 1 , 0 
device_name = utils.get_tf_device()
test_time = time.time() 
#prepare the character based tokenizer
tk, MAX_INDEX = tf_models.prepare_tokenizer(maxlen = tf_initialize.MAX_STRING_LENGTH)
X, y, class_list, classes = tf_models.text_2_sequence(unknown_df, tk, tf_initialize.MAX_STRING_LENGTH)

X_test = X
y_test = y

Highnam

In [ ]:

#start testing
total_stats = {}
#with tf.device(device_name):
model_name = 'highnam.h5'
high_model = tf_models.build_highnam()

#load the pretrained model    
high_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
high_predictions, high_test_time = test_model(high_trained_model, X_test)
high_y_pred = np.where(high_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
high_conf_matrix = confusion_matrix(y_test, high_y_pred)

#Accuracy
high_acc = accuracy_score(y_test, high_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  high_tag_auroc = roc_auc_score(y_test, high_y_pred)
  print(f"{name}: {high_tag_auroc}")

#Compute  classification report 
high_class_rpt = classification_report(y_test, high_y_pred, digits=4)
print(high_conf_matrix)
print(high_acc)
print(high_class_rpt)

In [ ]:
scores = []
for i in high_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('Highnam', 'Unknown DGA', round(high_acc.item(),3), prec, fscore, recall, high_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(high_tag_auroc.item(),3))

MI

In [ ]:

#with tf.device(device_name):
model_name = 'lstm1_mi.h5'
m1_model = tf_models.build_LSTM_MI_model(tf_initialize.MAX_INDEX, tf_initialize.MAX_STRING_LENGTH)

#load the pretrained model    
m1_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
m1_predictions, m1_test_time = test_model(m1_trained_model, X_test)
m1_y_pred = np.where(m1_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
m1_conf_matrix = confusion_matrix(y_test, m1_y_pred)

#Accuracy
m1_acc = accuracy_score(y_test, m1_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  m1_tag_auroc = roc_auc_score(y_test, m1_y_pred)
  print(f"{name}: {m1_tag_auroc}")

#Compute  classification report 
m1_class_rpt = classification_report(y_test, m1_y_pred, digits=4)
print(m1_conf_matrix)
print(m1_acc)
print(m1_class_rpt)

In [ ]:
scores = []
for i in m1_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('LSTM1.MI', 'Unknown DGA', round(m1_acc.item(),3), prec, fscore, recall, m1_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(m1_tag_auroc.item(),3))

CBDC

In [ ]:

#with tf.device(device_name):
model_name = 'n-CDBC.h5'
CBDC_model = tf_models.build_CBDC()

#load the pretrained model    
CBDC_trained_model =  tf.keras.models.load_model(f'{root}models/{model_name}')
CBDC_predictions, CBDC_test_time = test_model(CBDC_trained_model, X_test)
CBDC_y_pred = np.where(CBDC_predictions > THRESHOLD, upper, lower)

#Confusion Matrix
CBDC_conf_matrix = confusion_matrix(y_test, CBDC_y_pred)

#Accuracy
CBDC_acc = accuracy_score(y_test, CBDC_y_pred)

#Compute AUROC
print("AUROC per tag")
for i, name in enumerate([LABEL_COLUMNS[1:]]):
  CBDC_tag_auroc = roc_auc_score(y_test, CBDC_y_pred)
  print(f"{name}: {CBDC_tag_auroc}")

#Compute  classification report 
CBDC_class_rpt = classification_report(y_test, CBDC_y_pred, digits=4)

print(CBDC_conf_matrix)
print(CBDC_acc)
print(CBDC_class_rpt)

In [ ]:
scores = []
for i in CBDC_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('n-CBDC', 'Unknown DGA', round(CBDC_acc.item(),3), prec, fscore, recall, CBDC_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(CBDC_tag_auroc.item(),3))

DGA Transformers

In [ ]:
checkpoint = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/logs/lightning_logs/domain_name/version_2/checkpoints/epoch=4-step=7815.ckpt'
THRESHOLD = 0.5
dga_trained_model = DGATransformer.load_from_checkpoint(
    checkpoint, 
    LABEL_COLUMNS = test_df.columns.tolist()[1:]
)
dga_trained_model.eval()
dga_trained_model.freeze()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dga_trained_model = dga_trained_model.to(device)
LABELS = unknown_df.columns.tolist()[1:]
test_dataset = DGADataset(
  unknown_df,
  tokenizer,
  max_token_len=pt_initialize.MAX_LENGTH,
  categories = LABELS
)

dga_predictions = []
dga_labels = []

t0 = time.time()
for item in tqdm(test_dataset):
  _, prediction = dga_trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  dga_predictions.append(prediction.flatten())
  dga_labels.append(item["labels"].int())

dga_test_time = utils.format_time(time.time() - t0)


dga_predictions = torch.stack(dga_predictions).detach().cpu()
dga_labels = torch.stack(dga_labels).detach().cpu()


In [ ]:


dga_acc = accuracy(dga_predictions, dga_labels, threshold=THRESHOLD)
dga_y_pred = dga_predictions.numpy()
y_true = dga_labels.numpy()

upper, lower = 1, 0
dga_y_pred = np.where(dga_y_pred > THRESHOLD, upper, lower)
dga_class_rpt = classification_report(y_true, dga_y_pred, digits=4)

dga_conf_matrix = confusion_matrix(y_true, dga_y_pred)

for i, name in enumerate(test_df.columns.tolist()[1:]):
  dga_tag_auroc = auroc(dga_predictions[:, i], dga_labels[:, i], pos_label=1)

print(dga_conf_matrix)
print(dga_acc)
print(dga_class_rpt)
print(dga_tag_auroc)

In [ ]:
scores = []
for i in dga_class_rpt.split(' '):
  if i !='':
    scores.append(i.strip())

prec = float(scores[-9])
recall = float(scores[-8])
fscore = float(scores[-7])
print(prec, fscore, recall)
add_stats('DGA-Transformer','Unknown DGA', round(dga_acc.item(),3), prec, fscore, recall, dga_test_time, test_df[test_df['benign']==0].shape[0], test_df[test_df['benign']==1].shape[0], round(dga_tag_auroc.item(),3))

Results

In [ ]:
stats_df = utils.result_df(training_stats)
stats_df

In [ ]:
stats_df.to_csv('/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/unknown-6-6-2022-3layers.csv')

# **Draw Figures**

In [ ]:
image_path = '/content/drive/MyDrive/Colab Notebooks/DGA_Version_70length/results/images'

#get base models values
high_fpr, high_tpr, high_roc_auc = get_tf_fpr_fnr(high_predictions, y_test)
m1_fpr, m1_tpr, m1_roc_auc = get_tf_fpr_fnr(m1_predictions, y_test)
CBDC_fpr, CBDC_tpr, CBDC_roc_auc = get_tf_fpr_fnr(CBDC_predictions, y_test)

#get DGA_transformer values
dga_fpr, dga_tpr, dga_roc_auc = get_pt_fpr_tpr(dga_predictions.numpy(), dga_labels.numpy())


In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["micro"],
    high_tpr["micro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["micro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["micro"],
    m1_tpr["micro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["micro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["micro"],
    CBDC_tpr["micro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["micro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["micro"],
    dga_tpr["micro"],
    label="DGA-Trans Micro area = {0:0.3f}".format(dga_roc_auc["micro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Micro) Unknown Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/unknown_micro.png",dpi=300)

In [ ]:
# Plot all ROC curves
plt.figure()
lw = 2
plt.plot(
    high_fpr["macro"],
    high_tpr["macro"],
    label="Highnam Micro area = {0:0.3f}".format(high_roc_auc["macro"]),
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_fpr["macro"],
    m1_tpr["macro"],
    label="LSTM1.MI Micro area = {0:0.3f}".format(m1_roc_auc["macro"]),
    color="darkorange",
    linestyle=":",
    linewidth=4
)

plt.plot(
    CBDC_fpr["macro"],
    CBDC_tpr["macro"],
    label="CBDC Micro area = {0:0.3f}".format(CBDC_roc_auc["macro"]),
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)

plt.plot(
    dga_fpr["macro"],
    dga_tpr["macro"],
    label="DGA-Trans Macro area = {0:0.3f}".format(dga_roc_auc["macro"]),
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC (Macro) Unknown Dataset")
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/unknown_macro.png",dpi=300)

In [ ]:
n_classes = len(test_df.columns.tolist()[1:])
# Plot all ROC curves
plt.figure()
lw = 2
for i in range(n_classes):
  plt.plot(
      high_fpr[0],
      high_tpr[0],
      label="Highnam AUROC = {0:0.3f}".format(high_tag_auroc),
      color="aqua",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      m1_fpr[0],
      m1_tpr[0],
      label="LSTM1.MI AUROC = {0:0.3f}".format(m1_tag_auroc),
      color="darkorange",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      CBDC_fpr[0],
      CBDC_tpr[0],
      label="CBDC AUROC = {0:0.3f}".format(CBDC_tag_auroc),
      color="cornflowerblue",
      linestyle=":",
      linewidth=4
  )

  plt.plot(
      dga_fpr[0],
      dga_tpr[0],
      label="DGA-Trans AUROC = {0:0.3f}".format(dga_tag_auroc),
      color="purple",
      linestyle=":",
      linewidth=4
  )

  plt.plot([0, 1], [0, 1], "k--", lw=lw)
  plt.xlim([0.0, 1.05])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate")
  plt.ylabel("True Positive Rate")
  plt.title("ROC (AUROC) Unknown Dataset")
  plt.legend(loc="lower right")
  plt.show()
plt.savefig(f"{image_path}/unknown_auroc.png",dpi=300)

In [ ]:
# Plot all Precision Recall curves
high_precision, high_recall, _ = precision_recall_curve(y_test, high_y_pred)
m1_precision, m1_recall, _ = precision_recall_curve(y_test, m1_y_pred)
CBDC_precision, CBDC_recall, _ = precision_recall_curve(y_test, CBDC_y_pred)
dga_precision, dga_recall, _ = precision_recall_curve(dga_labels.numpy(), dga_predictions.numpy())
len_benign = len(y_test[y_test==1])
len_dga = len(y_test[y_test==0])



plt.figure()
lw = 2

plt.plot(
    high_precision,
    high_recall,
    label="Highnam",
    color="aqua",
    linestyle=":",
    linewidth=4
)

plt.plot(
    m1_precision,
    m1_recall,
    label="LSTM1.MI",
    color="darkorange",
    linestyle=":",
    linewidth=4
)
plt.plot(
    CBDC_precision,
    CBDC_recall,
    label="n-CBDC",
    color="cornflowerblue",
    linestyle=":",
    linewidth=4
)
plt.plot(
    dga_precision,
    dga_recall,
    label="DGA-Transformer",
    color="purple",
    linestyle=":",
    linewidth=4
)

plt.plot([0, 1], [len_benign, len_benign], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve (UnknownDGA)')
plt.legend(loc="lower right")
plt.show()
plt.savefig(f"{image_path}/unknown_prec_recall.png",dpi=300)